### KNN Regression Model with RF + RFECV Feature Selection

In [1]:
import os

cwd = os.getcwd()
print(cwd)

/home/rc/capstone/notebooks/model-tests


In [2]:
change_cwd = os.chdir("../../models")
!pwd

/home/rc/capstone/models


In [120]:
!python knn_regression.py \
--data ../data/Key_indicator_districtwise.csv \
--target Infant_Mortality_Rate_Imr_Total_Person \
--id-cols State_Name State_District_Name \
--correlation 60 --test-size 0.15 --random-state 50 --outdir ../notebooks/model-tests/test-3

KNN Regression Model
📊 Dataset: (284, 644)
🎯 Target: 'Infant_Mortality_Rate_Imr_Total_Person' → 'Infant_Mortality_Rate_Imr_Total_Person'
🧹 Deduplicating: 284 → 284 samples

📈 Raw Dataset Summary
📊 Dataset Shape: (284, 644)
🎯 Target Column: 'Infant_Mortality_Rate_Imr_Total_Person'
🔍 Total Missing Null/NaN Values: 9,297 (5.08%)
🎯 Target Missing: 0 / 284 (0.0%)

📋 Top 10 Features with Missing Null/NaN Values:
------------------------------------------------------------
  Total_Fertility_Rate_Urban               |    284 (100.0%)
  Total_Fertility_Rate_Rural               |    284 (100.0%)
  Post_Neo_Natal_Mortality_Rate_Urban      |     93 ( 32.7%)
  Under_Five_Mortality_Rate_U5MR_Urban_Female |     92 ( 32.4%)
  Under_Five_Mortality_Rate_U5MR_Urban_Male |     92 ( 32.4%)
  Neo_Natal_Mortality_Rate_Urban           |     92 ( 32.4%)
  Infant_Mortality_Rate_Imr_Urban_Female   |     92 ( 32.4%)
  Infant_Mortality_Rate_Imr_Urban_Person   |     92 ( 32.4%)
  Under_Five_Mortality_Rate_U5MR_Urba